In [1]:
!pip install transformers
!pip install ipywidgets

In [2]:
#加载日志信息
import logging
logging.basicConfig(level=logging.ERROR)
# 从 transformers 引入 TFBertForSequenceClassification,BertTokenizer
from transformers import TFBertForSequenceClassification,BertTokenizer
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ModuleNotFoundError: No module named 'tensorflow'

In [3]:
def convert_example_to_feature(review):
  
    return tokenizer.encode_plus(review, 
                add_special_tokens = True, # 添加 [CLS], [SEP]
                max_length = max_length, # bert使用的最大文本长度
                pad_to_max_length = True, # 添加 [PAD] tokens
                return_attention_mask = True, 
            truncation=True
              )

def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
    return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label


In [13]:
def encode_examples(ds, limit=-1):
    # 建立模型使用的数据集
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
    if (limit > 0):
        ds = ds.take(limit)
  
    for index, row in ds.iterrows():
        review = row["Abstract"]
        label = row["y"]
        bert_input = convert_example_to_feature(review)
  
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append([label])
    return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)


In [14]:
def split_dataset(df):
    train_set, test_set = train_test_split(df, 
        stratify=df['Type'],
        test_size=0.2, 
        random_state=42)

    return train_set, test_set


In [16]:
if __name__ == '__main__': 

    # 参数
    data_path = "Leukemia.csv" # 数据路径
    model_path = "bert-base-uncased" #模型路径，建议预先下载

    max_length = 64
    batch_size = 64
    learning_rate = 2e-5
    number_of_epochs = 10
    num_classes = 4 # 类别数

    # 读数据
    df_raw = pd.read_csv(data_path)    
    abstract = df_raw['Abstract']
    label = df_raw['Type']
    
    # 转换标签
    df_label = pd.DataFrame({"Type":["AML", "ALL", "CML", "CLL"],"y":list(range(4))})
    df_raw = pd.merge(df_raw,df_label,on="Type",how="left")
    # 划分数据
    train_data, test_data = split_dataset(df_raw)
    
    print(len(train_data))
    print(len(test_data))
    print(test_data.keys())

    # 分词器
    tokenizer = BertTokenizer.from_pretrained(model_path)
    # 训练数据集
    ds_train_encoded = encode_examples(train_data).shuffle(50).batch(batch_size)

    # 测试数据集
    ds_test_encoded = encode_examples(test_data).batch(batch_size)

    # 模型初始化
    model = TFBertForSequenceClassification.from_pretrained(model_path, num_labels=num_classes)

    # 使用Adam作为优化器
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate,epsilon=1e-08, clipnorm=1)
    # 使用交叉熵作为损失函数
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
    # 训练模型
    bert_history = model.fit(ds_train_encoded, epochs=number_of_epochs, validation_data=ds_test_encoded)
    # 估算测试集效果
    print("# evaluate test_set:",model.evaluate(ds_test_encoded))
    

160
40
Index(['Type', 'Authors', 'Title', 'Year', 'Electronic-resource-num',
       'Abstract', 'Periodical', 'Paper_id', 'y'],
      dtype='object')


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
3/3 [==============================] - 23s 2s/step - loss: 1.3962 - accuracy: 0.2375 - val_loss: 1.3772 - val_accuracy: 0.2750
Epoch 2/10
3/3 [==============================] - 2s 646ms/step - loss: 1.3661 - accuracy: 0.3000 - val_loss: 1.3622 - val_accuracy: 0.4000
Epoch 3/10
3/3 [==============================] - 2s 644ms/step - loss: 1.3257 - accuracy: 0.4563 - val_loss: 1.3286 - val_accuracy: 0.3000
Epoch 4/10
3/3 [==============================] - 2s 642ms/step - loss: 1.2923 - accuracy: 0.4500 - val_loss: 1.2965 - val_accuracy: 0.3250
Epoch 5/10
3/3 [==============================] - 2s 643ms/step - loss: 1.2131 - accuracy: 0.6250 - val_loss: 1.2741 - val_accuracy: 0.4250
Epoch 6/10
3/3 [==============================] - 2s 646ms/step - loss: 1.1567 - accuracy: 0.6313 - val_loss: 1.2026 - val_accuracy: 0.5000
Epoch 7/10
3/3 [==============================] - 2s 645ms/step - loss: 1.0926 - accuracy: 0.7250 - val_loss: 1.1245 - val_accuracy: 0.6750
Epoch 8/10
3/3 [======

In [17]:
from sklearn.metrics import confusion_matrix

# 得到模型预测结果
res = model.predict(ds_test_encoded)
pre = np.argmax(res["logits"], axis=1)
true = test_data['y']
# 生成混淆矩阵
cf = confusion_matrix(true, pre)
print(cf)

[[8 0 0 2]
 [0 8 1 1]
 [6 0 4 0]
 [1 0 0 9]]
